In [17]:
pip install imutils

  Using cached imutils-0.5.4-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
from IPython.display import display, Image
import PIL

import imutils

In [4]:
from ultralytics import YOLO
import cv2

from sort.sort import *
from util import get_car, read_license_plate, write_csv, interpolate_bounding_boxes, draw_border
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd
import ast

/Users/syshin/Desktop/Syshin/Men-in-Black/Automatic License Plate Recognition/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Using CPU. Note: This module is much faster with a GPU.


In [5]:
def show_image(img, display_size=(640, 480)):
    # Resize the image to the display size
    img_resized = cv2.resize(img, display_size)

    # Encode the resized image
    _, encoded_image = cv2.imencode('.png', img_resized)
    display(Image(data=encoded_image.tobytes()))

In [16]:
def process_and_visualize_license_plate(frame, license_plate_crop, show_image_function):
    """
    Process the license plate in the frame and visualize the steps.

    Args:
    frame (numpy.ndarray): The original frame from the video.
    bbox (tuple): The bounding box coordinates (x1, y1, x2, y2) for the license plate.
    show_image_function (function): Function to display the images.
    """

    if license_plate_crop.size > 0:
        # Convert to grayscale
        license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur
        license_plate_crop_blur = cv2.GaussianBlur(license_plate_crop_gray, (5, 5), 0)
        
        # Noise Reduction
        license_plate_crop_bfilter = cv2.bilateralFilter(license_plate_crop_blur, 11, 17, 17)

        # Apply Canny edge detection
        license_plate_crop_canny = cv2.Canny(license_plate_crop_bfilter, 75, 200)

        # Find contours and crop the image
        keypoints = cv2.findContours(frame.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(keypoints)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

        location = None
        for contour in contours:
            approx = cv2.approxPolyDP(contour, 10, True)
            if len(approx) == 4:
                location = approx
                break

        cropped_image = None
        if location is not None:
            mask = np.zeros(license_plate_crop_gray.shape, np.uint8)
            new_image = cv2.drawContours(mask, [location], 0, 255, -1)
            new_image = cv2.bitwise_and(license_plate_crop_gray, license_plate_crop_gray, mask=mask)

            (x, y) = np.where(mask == 255)
            if len(x) > 0 and len(y) > 0:
                (x1, y1) = (np.min(x), np.min(y))
                (x2, y2) = (np.max(x), np.max(y))
                cropped_image = new_image[x1:x2+1, y1:y2+1]
        
        # Visualize the processed images
        show_image_function(license_plate_crop)
        show_image_function(license_plate_crop_gray)
        show_image_function(license_plate_crop_canny)
        # show_image_function(new_image)
        # show_image_function(cropped_image)
        if cropped_image is not None:
            show_image_function(cropped_image)
        
        return license_plate_crop_canny

In [17]:
results = {}

mot_tracker = Sort()

# load models
coco_model = YOLO('yolov8n.pt')
# license_plate_detector = YOLO('models/11-10_best_weight_yolov8m_140_epochs.pt')
license_plate_detector = YOLO('models/11-08_best_weight_yolov8m_50_epoch.pt')

# load video
video_path = './sample/IMG_3359.mp4'
video_name = os.path.splitext(os.path.basename(video_path))[0]
cap = cv2.VideoCapture(video_path)

person = [0]
vehicles = [1, 2, 3, 5, 7]
traffic_signs = [9, 11]

class_id_dict = {
    0: 'person',
    1: 'bicycle',
    2: 'car',
    3: 'motorbike',
    5: 'bus',
    7: 'truck',
    9: 'traffic light',
    11: 'stop sign'
}

# read frames
frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = license_plate_detector(frame)[0]
        # print('detected license plate:', license_plates)
        for license_plate in license_plates.boxes.data.tolist():
            print('detected license plate boxes:', license_plate)
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)
            
            print(car_id)
            if car_id != -1:
                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate and visualize
                processed_img = process_and_visualize_license_plate(frame, license_plate_crop, show_image)
                
                # Check if processed_img is valid before calling read_license_plate
                if processed_img is not None:
                    license_plate_text, license_plate_text_score = read_license_plate(processed_img)
                    print('license_plate_text:', license_plate_text)
                
                    if license_plate_text is not None:
                        results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                      'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                        'text': license_plate_text,
                                                                        'bbox_score': score,
                                                                        'text_score': license_plate_text_score}}

                    
# Release resources
cap.release()
cv2.destroyAllWindows()

print("results: \n", results)



0: 384x640 2 cars, 100.6ms
Speed: 9.5ms preprocess, 100.6ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 164.6ms
Speed: 1.6ms preprocess, 164.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


detected license plate boxes: [0.0, 1096.78857421875, 368.0877685546875, 1293.8975830078125, 0.7526004314422607, 0.0]
-1
detected license plate boxes: [1868.941162109375, 1505.10498046875, 2369.58251953125, 1666.4091796875, 0.7486537098884583, 0.0]
12.0


error: OpenCV(4.8.1) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/contours.cpp:197: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'


In [ ]:
# write results
csv_file_name = f'./sample/{video_name}.csv'
write_csv(results, csv_file_name)

# Add missing data

# Load the CSV file
with open(csv_file_name, 'r') as file:
    reader = csv.DictReader(file)
    data = list(reader)

# Interpolate missing data
interpolated_data = interpolate_bounding_boxes(data)

# Write updated data to a new CSV file
header = ['frame_nmr', 'car_id', 'car_bbox', 'license_plate_bbox', 'license_plate_bbox_score', 'license_number', 'license_number_score']
with open(f'./sample/{video_name}_interpolated.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    writer.writerows(interpolated_data)


# Visualize

results = pd.read_csv(f'./sample/{video_name}_interpolated.csv')

# load video
cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(f'./sample/{video_name}_out.mp4', fourcc, fps, (width, height))

license_plate = {}
for car_id in np.unique(results['car_id']):
    max_ = np.amax(results[results['car_id'] == car_id]['license_number_score'])
    license_plate[car_id] = {'license_crop': None,
                             'license_plate_number': results[(results['car_id'] == car_id) &
                                                             (results['license_number_score'] == max_)]['license_number'].iloc[0]}
    cap.set(cv2.CAP_PROP_POS_FRAMES, results[(results['car_id'] == car_id) &
                                             (results['license_number_score'] == max_)]['frame_nmr'].iloc[0])
    ret, frame = cap.read()

    x1, y1, x2, y2 = ast.literal_eval(results[(results['car_id'] == car_id) &
                                              (results['license_number_score'] == max_)]['license_plate_bbox'].iloc[0].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

    license_crop = frame[int(y1):int(y2), int(x1):int(x2), :]
    license_crop = cv2.resize(license_crop, (int((x2 - x1) * 400 / (y2 - y1)), 400))

    license_plate[car_id]['license_crop'] = license_crop


frame_nmr = -1

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# read frames
ret = True
while ret:
    ret, frame = cap.read()
    frame_nmr += 1
    if ret:
        df_ = results[results['frame_nmr'] == frame_nmr]
        for row_indx in range(len(df_)):
            # draw car
            car_x1, car_y1, car_x2, car_y2 = ast.literal_eval(df_.iloc[row_indx]['car_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            draw_border(frame, (int(car_x1), int(car_y1)), (int(car_x2), int(car_y2)), (0, 255, 0), 25,
                        line_length_x=200, line_length_y=200)

            # draw license plate
            x1, y1, x2, y2 = ast.literal_eval(df_.iloc[row_indx]['license_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 12)

            # crop license plate
            license_crop = license_plate[df_.iloc[row_indx]['car_id']]['license_crop']

            H, W, _ = license_crop.shape

            try:
                frame[int(car_y1) - H - 100:int(car_y1) - 100,
                      int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = license_crop

                frame[int(car_y1) - H - 400:int(car_y1) - H - 100,
                      int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = (255, 255, 255)

                (text_width, text_height), _ = cv2.getTextSize(
                    license_plate[df_.iloc[row_indx]['car_id']]['license_plate_number'],
                    cv2.FONT_HERSHEY_SIMPLEX,
                    4.3,
                    17)

                cv2.putText(frame,
                            license_plate[df_.iloc[row_indx]['car_id']]['license_plate_number'],
                            (int((car_x2 + car_x1 - text_width) / 2), int(car_y1 - H - 250 + (text_height / 2))),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            4.3,
                            (0, 0, 0),
                            17)

            except:
                pass

        out.write(frame)
        frame = cv2.resize(frame, (1280, 720))

        # cv2.imshow('frame', frame)
        # cv2.waitKey(0)

out.release()
cap.release()

In [6]:
import cv2

# Replace with the path to your test image
image_path = frame

# Read the image
image = cv2.imread(image_path)

# Check if the image was loaded correctly
if image is not None:
    # Display the image in a window
    cv2.imshow('Test Image', image)

    # Wait for a key press before closing the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Error: Image not found or OpenCV not installed correctly.")


TypeError: Can't convert object to 'str' for 'filename'